# Tsirelson bound for the CHSH inequality

We consider the CHSH inequality:

$$S_\text{CHSH} = \langle A_0 B_0 \rangle + \langle A_0 B_1 \rangle + \langle A_1 B_0 \rangle - \langle A_1 B_1 \rangle.$$



We start by importing the following libraries

In [6]:
import numpy as np
import ncpol2sdpa as ncp
from ncpol2sdpa import *

We define the settings and the operators

In [7]:
# for the other inputs, especially when you give a large NPA level).
A_config = [2,2]
B_config = [2,2]

# Operators in problem
A = [Ai for Ai in ncp.generate_measurements(A_config, 'A')]
B = [Bj for Bj in ncp.generate_measurements(B_config, 'B')]

We define functions for adding extra monomials to the relaxation:

In [8]:
def get_extra_monomials():
    """
    Returns additional monomials to add to sdp relaxation.
    """

    monos = []

    Aflat = ncp.flatten(A)
    Bflat = ncp.flatten(B)

    # ABB
    for a in Aflat:
        for b in Bflat:
            for b2 in Bflat:
                monos += [a*b*b2]
    # AAB
    for a in Aflat:
        for a2 in Aflat:
            for b in Bflat:
                monos += [a*a2*b]

    #AB
    for a in Aflat:
        for b in Bflat:
            monos += [a*b]
    
    
    return monos[:]

def get_subs():
    """
    Returns any substitution rules to use with ncpol2sdpa. E.g. projections and
    commutation relations.
    """
    subs = {}
    # Get Alice and Bob's projective measurement constraints
    subs.update(ncp.projective_measurement_constraints(A,B))

    return subs

We are now ready to perform the relaxation and get the Tsirelson bound!

In [10]:
substitutions = get_subs()             # substitutions used in ncpol2sdpa
moment_ineqs = []                      # moment inequalities
moment_eqs = []                        # moment equalities
op_eqs = []                            # operator equalities
op_ineqs = []                          # operator inequalities
extra_monos = get_extra_monomials()    # extra monomials
LEVEL = 1

# S_W =  (2*A_{0|0}*B_{0|0} + 3 - 2*A0 - 2*B0 + 2*A_{0|0}*B_{0|1} + 2*A1*B0  - 2*A1*B1)/4

# Bounds on the parameters of sys
ops = ncp.flatten([A,B])        # Base monomials involved in problem
obj = ((2*A[0][0]*B[0][0] + 3 - 2*A[0][0] - 2*B[0][0] + 2*A[0][0]*B[1][0] + 2*A[1][0]*B[0][0] - 2*A[1][0]*B[1][0])/4)*8-4
sdp = ncp.SdpRelaxation(ops, verbose = 1, normalized=True, parallel=0)
sdp.get_relaxation(level = LEVEL,
                    equalities = op_eqs[:],
                    inequalities = op_ineqs[:],
                    momentequalities = moment_eqs[:],
                    momentinequalities = moment_ineqs[:],
                    objective = -obj,
                    substitutions = substitutions,
                    extramonomials = extra_monos)
sdp.solve('mosek')
print(-sdp.primal)

The problem has 4 noncommuting Hermitian variables
Calculating block structure...
Estimated number of SDP variables: 152
Generating moment matrix...
Reduced number of SDP variables: 40 40 (done: 100.66%, ETA 00:00:-0.0)

Problem
  Name                   :                 
  Objective sense        : minimize        
  Type                   : CONIC (conic optimization problem)
  Constraints            : 40              
  Affine conic cons.     : 0               
  Disjunctive cons.      : 0               
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 1 (scalarized: 153)
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 1                 time                   : 0.00            
Lin. dep.  - tries    

Our numerical calculation is quite close to the analytical bound!

In [11]:
2*np.sqrt(2)

np.float64(2.8284271247461903)

We can also write the matrix to a csv file:

In [ ]:
sdp.write_to_file("CHSH.csv")